# Creating a model and training it for generating and clustering music

In [1]:
#TODO: Create models: VAE, WAE, U-NET
#Save model weights
#Evaluate in this file?

In [2]:
#libraries
import deeplay as dl
import deeptrack as dt
import torch

In [ ]:
#VAE
vae = dl.VariationalAutoEncoder(
    latent_dim=2, channels=[32, 64],
    reconstruction_loss=torch.nn.BCELoss(reduction="sum"), beta=1,
).create()

print(vae)

In [ ]:
#WAE
wae = dl.WassersteinAutoEncoder(
    channels=[32, 64, 128], latent_dim=20,
    reconstruction_loss=torch.nn.MSELoss(reduction="mean"),
).create()

print(wae)